In [1]:
%%html
<style>
table {float:left}
</style>

# <font color="darkgray">Classical Reservoir Computing:</font><br>Mathematical Foundations

<table>
    <tr><td><strong>Aim:</strong></td>
        <td>To formulate a classical reservoir computing model, so that it could be implemented 
            in <strong>Python</strong>.</tr>
    <tr><td><strong>Author:</strong></td>
        <td>Jacob L. Cybulski (<a href="https://jacobcybulski.com/" target="_blank">website</a>),
            <em>Enquanted</em></td></tr>
    <tr><td><strong>Release:</strong></td>
        <td>April 2025</td></tr>
    <tr><td><strong>References:</strong></td>
        <td>Unfortunately, there is no single simple formulation of a classical reservoir. 
            Numerous papers have been checked for some good insights, including:
            <ul>
            <li><a href = "https://doi.org/10.1103/PhysRevApplied.14.024065" target="_blank">
                Chen, J., Nurdin, H.I., Yamamoto, N., 2020. Temporal Information Processing on Noisy Quantum Computers. Phys. Rev. Applied 14, 024065.
</a></li>
            <li><a href = "https://doi.org/10.1088/2634-4386/ac7db7" target="_blank">Cucchi, M., Abreu, S., Ciccone, G., Brunner, D., Kleemann, H., 2022. Hands-on reservoir computing: a tutorial for practical implementation. Neuromorph. Comput. Eng. 2, 032002. 
</a></li>
            <li><a href = "https://doi.org/10.1038/s41467-021-25801-2" target="_blank">Gauthier, D.J., Bollt, E., Griffith, A., Barbosa, W.A.S., 2021. Next generation reservoir computing. Nat Commun 12, 5564.
</a></li>
        </ul>
        And many others...</td>
    </tr>
    <tr><td><strong>License:</strong></td>
        <td>This project is licensed under the
            <a href="./LICENSE" target="_blank">GNU General Public License V3</a></td></tr>
</table>

### Reservoir computing (RC)
RC is a machine learning model that integrates elements of a family of temporal neural networks, such as recurrent neural networks, liquid-state machines and echo-state networks.
While RCs can be applied to all types of data, they are especially useful for the processing of temporal data. 
The reservoir computing architecture centers around a large sparse neural network of randomly initialised and fixed weights, called a (_**reservoir**_), 
which is responsible for transforming input into a higher-dimensional space.
In high-dimensional space, the transformed input can then be used to train a simple linear model (_**readout layer**_) 
to separate (e.g. for the purpose of _**classification**_) the reservoir states with relative ease.
The _**reservoir dynamics**_ can be specified by a set of differential equations (_**update rules**_)
responsible for describing changes to the reservoir state over time.

Typical reservoir computing applications include: time-series forecasting, speech recognition and video analysis, control of robots or autonomous vehicles, as well as, predicting weather patterns and stock markets.
It is also worth noting that RC concepts also apply to physical systems that are continuous in space and time.

This is Jacob's take on the RC working!

### 1. Reservoir dynamics

The reservoir needs to be designed to have the following two properties, e.g.
- **Echo state property (ESP)**: The reservoir should be able to retain (or "echo") past information for a short period of time.
- **Fading memory**: The influence of the past inputs on the reservoir state should fade over time.

Therefore the reservoir dynamics can be described as follows:
$$
     \mathbf{r}(t+1) = f(\mathbf{W}_{\text{in}} \mathbf{u}(t) + \mathbf{W}_{\text{res}} \mathbf{r}(t))
$$
where:
- $\mathbf{r}(t)$ is the reservoir state at time $t$,
- $\mathbf{u}(t)$ is the input at time $t$,
- $\mathbf{W}_{\text{in}}$ is a matrix transforming input to the reservoir weight matrix  (fixed, small and random),
- $\mathbf{W}_{\text{res}}$ is the reservoir weight matrix (sparse, fixed and random),
- $f$ is a nonlinear activation function (e.g., $\tanh$).

### 2. Reservoir Weight Matrix

The reservoir weight matrix $\mathbf{W}_{\text{res}}$ should be sparse and randomly initialized.
To ensure that $\mathbf{W}_{\text{res}}$ does not have very large or very small weights, which may prevent the short-term memory stability (ESP) or fading long-term memory, we scale the reservoir weights by the **spectral radius** $\rho(\mathbf{W}_{\text{res}})$ (the largest absolute eigenvalue):
$$
       \mathbf{W}_{\text{res}} = \frac{\alpha}{\rho(\mathbf{W}_{\text{res}})} \mathbf{W}_{\text{res}}
$$
       where $\alpha$ is a scaling factor (typically $0 < \alpha < 1$, e.g., 0.9).

### 3. Readout layer

The readout layer is a simple (usually) linear model that maps the reservoir's high-dimensional states to the target output, which at time $t$ is:
$$\mathbf{y}(t) = \mathbf{W}_{\text{out}} \mathbf{r}(t)$$
where:
 - $\mathbf{y}(t)$ is the output,
 - $\mathbf{W}_{\text{out}}$ is the output weight matrix (the only trainable part of the system),
 - $\mathbf{r}(t)$ is the reservoir state at time $t$.

The goal is to find the optimal $\mathbf{W}_{\text{out}}$ such that the output $\mathbf{y}(t)$ matches the target as closely as possible.

### 4. Training the readout layer

Training involves collecting the reservoir states $\mathbf{r}(t)$ over time and using them to solve for $\mathbf{W}_{\text{out}}$.

Let us define $\mathbf{R}$ and $\mathbf{Y}$ as follows:
   - Let $\mathbf{R} \in \mathbb{R}^{N \times T}$ be the matrix of reservoir states (where $ N $ is the number of reservoir "neurons" and $ T $ is the number of time steps).
   - Let $\mathbf{Y} \in \mathbb{R}^{d \times T}$ be the matrix of target outputs (where $ d $ is the output dimension).


The process involves the following steps:
1. **Drive the Reservoir**: Feed the input data $\mathbf{u}(t)$ into the reservoir and collect the corresponding reservoir states $\mathbf{r}(t)$ over time.
1. **Collect Data**: Stack the reservoir states into a matrix $\mathbf{R}$ and the target outputs into a matrix $\mathbf{Y}$.
1. **Solve for $\mathbf{W}_{\text{out}}$** by using _**ridge regression**_ (see step 7 for explanation):

$$\mathbf{W}_{\text{out}} = \mathbf{Y} \mathbf{R}^T (\mathbf{R} \mathbf{R}^T + \lambda \mathbf{I})^{-1}$$

where $\lambda$ is a regularization parameter (used in ridge regression to prevent overfitting).

### 5. Leaky integration
We can optionally introduce a leakage rate $\gamma$ (e.g., 0.1 to 0.5) to control the speed of the reservoir dynamics. In this case, the state update equation becomes:
$$
     \mathbf{r}(t+1) = (1-\gamma) \mathbf{r}(t) + \gamma f(\mathbf{W}_{\text{in}} \mathbf{u}(t) + \mathbf{W}_{\text{res}} \mathbf{r}(t))
$$
     This helps balance fast and slow dynamics in the reservoir.

### 6. Test the model

For each input sequence in the test set, 
it is possible to use the trained readout model to predict the next value. 
This can be achieved by the following steps:

1. Update the reservoir state $\mathbf{r}(t)$ (as in training).
1. Compute the predicted output:
    $$
       \hat{y}(t) = \mathbf{W}_{\text{out}} \mathbf{r}(t)
    $$
1. Compare the predicted output $\hat{y}(t)$ to the true value $y(t)$, and possibly
measure the prediction performance using metrics such as **Mean Squared Error (MSE)**:
    $$
       \text{MSE} = \frac{1}{N} \sum_{t=1}^N (y(t) - \hat{y}(t))^2
    $$

### 7. Why ridge regression is a solution for $W_{out}$

Our objective is to:
   - Find $ \mathbf{W}_{\text{out}} \in \mathbb{R}^{d \times N} $ that minimizes the **mean squared error (MSE)** with **L2 regularization** (ridge penalty):
$$
     \mathcal{L}(\mathbf{W}_{\text{out}}) = \|\mathbf{Y} - \mathbf{W}_{\text{out}} \mathbf{R}\|_F^2 + \lambda \|\mathbf{W}_{\text{out}}\|_F^2,
$$
     where $ \lambda \geq 0 $ is the regularization coefficient, and $ \|\cdot\|_F $ is the Frobenius norm.

Let us use the ***Frobenius norm*** as a loss function. Its advantage in machine learning is that it measures squared error between expected and predicted matrix operation, it is a generalisation of Euclidean (L2) norm, it is differentiable and convex, and can be regularised.<br>
It is defined as 
$$ 
     \|\mathbf{A}\|_F^2 = \text{tr}(\mathbf{A}^\top \mathbf{A}) 
$$
     So now:
     
$$
   \mathcal{L}(\mathbf{W}_{\text{out}}) = \text{tr}\left[ (\mathbf{Y} - \mathbf{W}_{\text{out}} \mathbf{R})^\top (\mathbf{Y} - \mathbf{W}_{\text{out}} \mathbf{R}) \right] + \lambda \, \text{tr}(\mathbf{W}_{\text{out}}^\top \mathbf{W}_{\text{out}}).
$$


Let us take the gradient w.r.t. $ \mathbf{W}_{\text{out}} $:
$$
   \nabla_{\mathbf{W}_{\text{out}}} \mathcal{L} = -2 (\mathbf{Y} - \mathbf{W}_{\text{out}} \mathbf{R}) \mathbf{R}^\top + 2 \lambda \mathbf{W}_{\text{out}}.
$$

To identify the minimum we find the zero gradient:
$$
   -2 (\mathbf{Y} - \mathbf{W}_{\text{out}} \mathbf{R}) \mathbf{R}^\top + 2 \lambda \mathbf{W}_{\text{out}} = 0.
$$
    which can be simplified as:
$$
   \mathbf{Y} \mathbf{R}^\top = \mathbf{W}_{\text{out}} (\mathbf{R} \mathbf{R}^\top + \lambda \mathbf{I}).
$$

When $ \lambda > 0 $, we can now solve for $ \mathbf{W}_{\text{out}} $, assuming $ (\mathbf{R} \mathbf{R}^\top + \lambda \mathbf{I}) $ is invertible:
$$
   \mathbf{W}_{\text{out}} = \mathbf{Y} \mathbf{R}^\top (\mathbf{R} \mathbf{R}^\top + \lambda \mathbf{I})^{-1}.
$$

   This is the **ridge regression solution** for the output weights in reservoir computing.

When $ \lambda = 0 $ (no regularisation), we are dealing with **ordinary least squares**, in which case the solution reduces to pseudoinverse:
$$
\mathbf{W}_{\text{out}} = \mathbf{Y} \mathbf{R}^\top (\mathbf{R} \mathbf{R}^\top)^{-1} = \mathbf{Y} \mathbf{R}^+,
$$
where $ \mathbf{R}^+ $ is the Moore-Penrose pseudoinverse of $ \mathbf{R} $.